In [5]:
; cl-waffeを読み込む
(load "../../cl-waffe.asd")
(ql:quickload :cl-waffe :silent t)
(use-package :cl-waffe)

T

(:CL-WAFFE)

T

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining PERFORM (#<STANDARD-CLASS ASDF/LISP-ACTION:TEST-OP>
                                                           #<SB-MOP:EQL-SPECIALIZER #<SYSTEM "cl-waffe/test">>) in DEFMETHOD


### 行列の初期化
関数`(!randn dim)`は平均0分散1の標準分布をサンプリングし、与えられた次元数`dim`の行列を返します。

In [18]:
(!randn `(10 10))

#Const(((1.334... 0.577... ~ 0.900... 1.191...)        
                 ...
        (0.810... 1.523... ~ -1.18... -1.55...)) :mgl t :shape (10 10) :backward NIL)

### 変数を定義する。
上記の関数で生成された行列は、Const(定数)と見做され、逆伝播時に勾配を生成しません。

マクロ`(parameter tensor)`を介して受け取った定数を変数にします。

parameterマクロの前後で計算ノードは途切れることに注意してください。

In [19]:
(parameter *)

#Parameter{((1.334... 0.577... ~ 0.900... 1.191...)            
                         ...
            (0.810... 1.523... ~ -1.18... -1.55...)) :mgl t :shape (10 10) :backward NIL}

### 計算ノードを構築する

マクロ`(with-no-grad &body body)`内部でない限り、cl-waffeのノードを介した計算は計算ノードを構築します。

計算ノードが構築されていたら、TensorをPrintした時に`:backward ノード名`と表示されます。

In [26]:
(defparameter a (!randn `(3 3)))

(let ((result (!add a 0.0)))
     (print result)
     (print (cl-waffe::waffetensor-state result)))
nil

A

<Node: ADDTENSOR{W1159}>

NIL


#Const(((-0.15... -0.03... 0.805...)        
                 ...
        (-0.28... -0.47... -1.41...)) :mgl t :shape (3 3) :backward <Node: ADDTENSOR{W1159}>) 
<Node: ADDTENSOR{W1159}> 

## 順伝播と逆伝播

線形回帰モデル
$$
y=Ax+b
$$
を計算する。

構築された計算ノードは、最後のノードを`(backward out)`のように呼び出すことで逆伝播される。

逆伝播はスカラー値に対してでないと定義されないため、最後の計算ノードは損失関数や、`!sum`, `!mean`等になる。

各変数は`(grad tensor)`で勾配を取り出される。（勾配はTensorのデータ構造に依る、構造体`WaffeTensor`ではないことに注意）

In [28]:
(defparameter weight (parameter (!randn `(3 3))))
(defparameter x      (parameter (!randn `(3 3))))

(defparameter bias   (parameter (!randn `(1 3))))

(let ((out (!add (!matmul weight x) bias)))
     (print out)
     
     (setq out (!sum out))
     
     (backward out)
     
     (print (grad weight))
     (print (grad x))
     (print (grad bias)))

WEIGHT

X

BIAS

#<MGL-MAT:MAT 1x3 AF #2A((0.11111112 0.11111112 0.11111112))>


#Const(((2.938... 0.878... -0.29...)        
                 ...
        (1.264... -0.10... -0.60...)) :mgl t :shape (3 3) :backward <Node: BROADCASTINGADDTENSOR{W1167}>) 
#<MGL-MAT:MAT 3x3 F #2A((-0.22465025 -0.122844614 0.4362781)
                        (-0.22465025 -0.122844614 0.4362781)
                        (-0.22465025 -0.122844614 0.4362781))> 
#<MGL-MAT:MAT 3x3 F #2A((0.07945849 0.07945849 0.07945849)
                        (0.061249208 0.061249208 0.061249208)
                        (0.07049674 0.07049674 0.07049674))> 
#<MGL-MAT:MAT 1x3 F #2A((0.11111112 0.11111112 0.11111112))> 

## ノードを定義する

[defnode](https://hikettei.github.io/cl-waffe-docs/docs/cl-waffe.html#2-defnode)マクロを用いて計算ノードの順伝播と逆伝播を定義できます。

defnodeの詳細な解説は、[Qiitaの記事](https://qiita.com/hikettei/items/f38e0bba89795ec8bff9#%E8%87%AA%E5%8B%95%E5%BE%AE%E5%88%86)に置いてあります。